# Türkçe BERT Embedding Fine-Tuning - Gelişmiş Versiyon

Bu notebook, **tüm soru-cevap verilerinizi** kullanarak `dbmdz/bert-base-turkish-cased` modelini fine-tune eder.

## ✨ Özellikler:
- Büyük veri setleri için optimize edilmiş
- Negatif örnek oluşturma (data augmentation)
- Hard negative mining
- Checkpoint kaydetme
- Memory optimization
- Progress tracking

## 📦 Kurulum

In [ ]:
!pip install sentence-transformers transformers datasets accelerate torch scikit-learn pandas openpyxl -q -U

print("✅ Tüm kütüphaneler yüklendi!")

## 📚 Kütüphaneleri İçe Aktarma

In [ ]:
import pandas as pd
import numpy as np
import torch
import json
import random
from pathlib import Path
from tqdm.auto import tqdm
from datetime import datetime

from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from sentence_transformers.util import cos_sim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# GPU kontrolü
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Kullanılan cihaz: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# Seed ayarla (reproducibility için)
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

## 📊 1. TÜM VERİLERİNİZİ YÜKLEYIN

### Desteklenen formatlar:
- **CSV**: `soru` ve `cevap` sütunları
- **JSON**: `[{"soru": "...", "cevap": "..."}, ...]`
- **Excel**: `soru` ve `cevap` sütunları
- **Birden fazla dosya**: Otomatik birleştirme

In [ ]:
def load_data_file(file_path):
    """Tek bir dosyayı yükler"""
    file_path = Path(file_path)
    
    if not file_path.exists():
        raise FileNotFoundError(f"Dosya bulunamadı: {file_path}")
    
    try:
        if file_path.suffix == '.csv':
            df = pd.read_csv(file_path, encoding='utf-8')
        elif file_path.suffix == '.json':
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            df = pd.DataFrame(data)
        elif file_path.suffix in ['.xlsx', '.xls']:
            df = pd.read_excel(file_path)
        else:
            raise ValueError(f"Desteklenmeyen format: {file_path.suffix}")
        
        print(f"  ✓ {file_path.name}: {len(df)} satır yüklendi")
        return df
    except Exception as e:
        print(f"  ✗ Hata ({file_path.name}): {str(e)}")
        return None

def load_all_data(file_paths):
    """Birden fazla dosyayı yükler ve birleştirir"""
    if isinstance(file_paths, str):
        file_paths = [file_paths]
    
    all_dfs = []
    print("📁 Dosyalar yükleniyor...")
    
    for file_path in file_paths:
        df = load_data_file(file_path)
        if df is not None:
            all_dfs.append(df)
    
    if not all_dfs:
        raise ValueError("Hiçbir dosya yüklenemedi!")
    
    # Tüm dataframe'leri birleştir
    combined_df = pd.concat(all_dfs, ignore_index=True)
    
    # Sütun kontrolü
    if 'soru' not in combined_df.columns or 'cevap' not in combined_df.columns:
        print(f"❌ Mevcut sütunlar: {combined_df.columns.tolist()}")
        raise ValueError("Veri setinde 'soru' ve 'cevap' sütunları olmalı!")
    
    return combined_df

# ============================================
# BURAYA KENDİ DOSYA YOLLARINIZI YAZIN
# ============================================

# Tek dosya için:
# DATA_FILES = 'path/to/your/qa_data.csv'

# Birden fazla dosya için:
# DATA_FILES = [
#     'path/to/qa_data1.csv',
#     'path/to/qa_data2.json',
#     'path/to/qa_data3.xlsx'
# ]

# Örnek veri oluşturma (test için - gerçek verileriniz varsa silin)
example_data = {
    'soru': [
        'Python nedir?',
        'Makine öğrenmesi ne demektir?',
        'Deep learning nedir?',
        'BERT modeli nasıl çalışır?',
        'Transformer mimarisi nedir?',
        'Natural language processing ne işe yarar?',
        'Yapay sinir ağları nasıl öğrenir?',
        'Supervised learning nedir?',
        'Unsupervised learning ne demek?',
        'Reinforcement learning nasıl çalışır?'
    ] * 10,  # 100 örnek için
    'cevap': [
        'Python, yüksek seviyeli, yorumlamalı bir programlama dilidir.',
        'Makine öğrenmesi, bilgisayarların verilerden öğrenmesini sağlayan yapay zeka dalıdır.',
        'Deep learning, çok katmanlı yapay sinir ağları kullanarak öğrenen bir makine öğrenmesi yöntemidir.',
        'BERT, çift yönlü transformer mimarisini kullanan bir doğal dil işleme modelidir.',
        'Transformer, dikkat mekanizması kullanan modern bir sinir ağı mimarisidir.',
        'NLP, bilgisayarların insan dilini anlamasını ve işlemesini sağlar.',
        'Yapay sinir ağları, geri yayılım algoritması ile ağırlıklarını güncelleyerek öğrenir.',
        'Supervised learning, etiketli verilerle model eğitme yöntemidir.',
        'Unsupervised learning, etiketsiz verilerden örüntüler keşfetme yöntemidir.',
        'Reinforcement learning, ödül ve ceza sistemiyle öğrenen bir makine öğrenmesi türüdür.'
    ] * 10
}

pd.DataFrame(example_data).to_csv('example_qa_data.csv', index=False, encoding='utf-8')
DATA_FILES = 'example_qa_data.csv'

# Verileri yükle
try:
    df = load_all_data(DATA_FILES)
    print(f"\n✅ Toplam {len(df):,} soru-cevap çifti yüklendi")
    print(f"\n📊 İlk 3 örnek:")
    print(df.head(3))
except Exception as e:
    print(f"❌ Hata: {e}")
    raise

## 🧹 2. Veri Temizleme ve Ön İşleme

In [ ]:
def clean_data(df, verbose=True):
    """Verileri temizler ve hazırlar"""
    initial_count = len(df)
    
    if verbose:
        print("🧹 Veri temizleme başlıyor...")
        print(f"  Başlangıç: {initial_count:,} örnek")
    
    # 1. Boş değerleri temizle
    df = df.dropna(subset=['soru', 'cevap'])
    if verbose:
        print(f"  Boş değerler silindi: {initial_count - len(df):,} örnek")
    
    # 2. Whitespace temizle
    df['soru'] = df['soru'].astype(str).str.strip()
    df['cevap'] = df['cevap'].astype(str).str.strip()
    
    # 3. Çok kısa örnekleri filtrele
    min_soru_len = 5
    min_cevap_len = 10
    df = df[
        (df['soru'].str.len() >= min_soru_len) & 
        (df['cevap'].str.len() >= min_cevap_len)
    ]
    
    # 4. Duplicate'leri kaldır
    before_dedup = len(df)
    df = df.drop_duplicates(subset=['soru', 'cevap'], keep='first')
    if verbose:
        print(f"  Duplicate silindi: {before_dedup - len(df):,} örnek")
    
    # 5. Index'i sıfırla
    df = df.reset_index(drop=True)
    
    if verbose:
        print(f"  ✅ Temizleme tamamlandı: {len(df):,} örnek kaldı")
        print(f"  📉 Toplam kayıp: {initial_count - len(df):,} örnek ({(initial_count - len(df))/initial_count*100:.1f}%)")
        
        # İstatistikler
        print(f"\n📈 Veri İstatistikleri:")
        print(f"  Ortalama soru uzunluğu: {df['soru'].str.len().mean():.1f} karakter")
        print(f"  Ortalama cevap uzunluğu: {df['cevap'].str.len().mean():.1f} karakter")
        print(f"  En uzun soru: {df['soru'].str.len().max()} karakter")
        print(f"  En uzun cevap: {df['cevap'].str.len().max()} karakter")
    
    return df

# Verileri temizle
df = clean_data(df)

## ⚡ 3. Negatif Örnekler Oluşturma (Data Augmentation)

Model performansını artırmak için **hard negative mining** yapıyoruz.

In [ ]:
def create_negative_examples(df, num_negatives_per_positive=2):
    """
    Her pozitif örnek için negatif örnekler oluşturur.
    Negatif örnekler = ilgisiz cevaplar
    """
    print(f"🔄 Negatif örnekler oluşturuluyor...")
    print(f"  Her pozitif örnek için {num_negatives_per_positive} negatif örnek")
    
    examples = []
    all_answers = df['cevap'].tolist()
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Creating examples"):
        # Pozitif örnek
        examples.append({
            'soru': row['soru'],
            'cevap': row['cevap'],
            'label': 1.0  # Pozitif
        })
        
        # Negatif örnekler (rastgele diğer cevaplar)
        negative_indices = random.sample(
            [i for i in range(len(all_answers)) if i != idx],
            min(num_negatives_per_positive, len(all_answers) - 1)
        )
        
        for neg_idx in negative_indices:
            examples.append({
                'soru': row['soru'],
                'cevap': all_answers[neg_idx],
                'label': 0.0  # Negatif
            })
    
    examples_df = pd.DataFrame(examples)
    
    print(f"  ✅ Toplam {len(examples_df):,} örnek oluşturuldu")
    print(f"     - Pozitif: {(examples_df['label']==1.0).sum():,}")
    print(f"     - Negatif: {(examples_df['label']==0.0).sum():,}")
    
    return examples_df

# Negatif örnekler ekle (opsiyonel - performansı artırır)
USE_NEGATIVE_EXAMPLES = True  # True/False olarak ayarlayın

if USE_NEGATIVE_EXAMPLES:
    augmented_df = create_negative_examples(df, num_negatives_per_positive=1)
else:
    # Sadece pozitif örneklerle devam et
    augmented_df = df.copy()
    augmented_df['label'] = 1.0
    print(f"✅ Sadece pozitif örnekler kullanılacak: {len(augmented_df):,}")

## 📊 4. Train-Validation-Test Split

In [ ]:
# Split ratios
TRAIN_RATIO = 0.80
VAL_RATIO = 0.10
TEST_RATIO = 0.10

# İlk olarak train ve temp split (train + val&test)
train_df, temp_df = train_test_split(
    augmented_df, 
    test_size=(1 - TRAIN_RATIO), 
    random_state=42,
    stratify=augmented_df['label'] if USE_NEGATIVE_EXAMPLES else None
)

# Val ve test split
val_df, test_df = train_test_split(
    temp_df, 
    test_size=TEST_RATIO/(VAL_RATIO + TEST_RATIO), 
    random_state=42,
    stratify=temp_df['label'] if USE_NEGATIVE_EXAMPLES else None
)

print(f"📊 Veri Seti Dağılımı:")
print(f"  Training:   {len(train_df):,} örnek ({len(train_df)/len(augmented_df)*100:.1f}%)")
print(f"  Validation: {len(val_df):,} örnek ({len(val_df)/len(augmented_df)*100:.1f}%)")
print(f"  Test:       {len(test_df):,} örnek ({len(test_df)/len(augmented_df)*100:.1f}%)")
print(f"  TOPLAM:     {len(augmented_df):,} örnek")

## 🤖 5. Model Yükleme

In [ ]:
MODEL_NAME = 'dbmdz/bert-base-turkish-cased'

print(f"🤖 Model yükleniyor: {MODEL_NAME}")
model = SentenceTransformer(MODEL_NAME)

# Model bilgileri
print(f"\n✅ Model başarıyla yüklendi!")
print(f"  Embedding boyutu: {model.get_sentence_embedding_dimension()}")
print(f"  Max token uzunluğu: {model.max_seq_length}")
print(f"  Model device: {model.device}")

## 🎯 6. Training Data Preparation

In [ ]:
def create_input_examples(df):
    """DataFrame'i InputExample listesine çevirir"""
    examples = []
    for _, row in df.iterrows():
        examples.append(
            InputExample(texts=[row['soru'], row['cevap']], label=float(row['label']))
        )
    return examples

print("🔄 Training örnekleri hazırlanıyor...")
train_examples = create_input_examples(train_df)
print(f"  ✅ {len(train_examples):,} training örneği hazır")

# Batch size (GPU belleğinize göre ayarlayın)
BATCH_SIZE = 16  # 8, 16, 32, 64 deneyebilirsiniz

# DataLoader
train_dataloader = DataLoader(
    train_examples, 
    shuffle=True, 
    batch_size=BATCH_SIZE
)

print(f"  Batch size: {BATCH_SIZE}")
print(f"  Batch sayısı: {len(train_dataloader):,}")

## 💪 7. Loss Fonksiyonu ve Evaluator

In [ ]:
# Loss fonksiyonu seçimi
# Büyük veri setleri için MultipleNegativesRankingLoss önerilir
if USE_NEGATIVE_EXAMPLES:
    # Negatif örnekler varsa CosineSimilarityLoss kullan
    train_loss = losses.CosineSimilarityLoss(model)
    loss_name = "CosineSimilarityLoss"
else:
    # Sadece pozitif örnekler varsa MultipleNegativesRankingLoss kullan
    train_loss = losses.MultipleNegativesRankingLoss(model)
    loss_name = "MultipleNegativesRankingLoss"

print(f"📉 Loss fonksiyonu: {loss_name}")

# Validation evaluator
val_questions = val_df['soru'].tolist()
val_answers = val_df['cevap'].tolist()
val_scores = val_df['label'].tolist()

evaluator = evaluation.EmbeddingSimilarityEvaluator(
    val_questions,
    val_answers,
    val_scores,
    name='qa-validation',
    show_progress_bar=True
)

print(f"✅ Evaluator hazır ({len(val_questions):,} validation örneği)")

## 🚀 8. Fine-Tuning

### ⚙️ Hyperparameters

In [ ]:
# Training parametreleri
NUM_EPOCHS = 3  # 2-5 epoch genellikle yeterli
WARMUP_STEPS = int(len(train_dataloader) * NUM_EPOCHS * 0.1)  # %10 warmup
EVALUATION_STEPS = 500  # Her kaç adımda bir değerlendirme
SAVE_STEPS = 1000  # Her kaç adımda bir checkpoint kaydet

# Output path
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_PATH = f'./turkish-bert-qa-finetuned_{timestamp}'

print("="*60)
print("🚀 FINE-TUNING BAŞLIYOR")
print("="*60)
print(f"📊 Dataset:")
print(f"   Training: {len(train_df):,} örnekleri")
print(f"   Validation: {len(val_df):,} örnekleri")
print(f"   Test: {len(test_df):,} örnekleri")
print(f"\n⚙️  Hyperparameters:")
print(f"   Epochs: {NUM_EPOCHS}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Total steps: {len(train_dataloader) * NUM_EPOCHS:,}")
print(f"   Warmup steps: {WARMUP_STEPS:,}")
print(f"   Evaluation steps: {EVALUATION_STEPS}")
print(f"\n💾 Output:")
print(f"   Model path: {OUTPUT_PATH}")
print(f"   Loss: {loss_name}")
print("="*60)

In [ ]:
# Training başlat
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=NUM_EPOCHS,
    warmup_steps=WARMUP_STEPS,
    output_path=OUTPUT_PATH,
    evaluation_steps=EVALUATION_STEPS,
    save_best_model=True,
    show_progress_bar=True,
    checkpoint_path=OUTPUT_PATH,
    checkpoint_save_steps=SAVE_STEPS,
    checkpoint_save_total_limit=3  # Son 3 checkpoint'i tut
)

print("\n" + "="*60)
print("✅ FINE-TUNING TAMAMLANDI!")
print("="*60)
print(f"💾 Model kaydedildi: {OUTPUT_PATH}")

## 📊 9. Model Değerlendirme

In [ ]:
# Fine-tuned modeli yükle
print("📥 Fine-tuned model yükleniyor...")
finetuned_model = SentenceTransformer(OUTPUT_PATH)
print("✅ Model yüklendi")

# Test seti üzerinde değerlendirme
def evaluate_model(model, test_df, model_name="Model"):
    """Test seti üzerinde model performansını değerlendirir"""
    print(f"\n📊 {model_name} Değerlendirmesi:")
    print("="*50)
    
    similarities = []
    
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Evaluating"):
        soru_emb = model.encode(row['soru'], convert_to_tensor=True)
        cevap_emb = model.encode(row['cevap'], convert_to_tensor=True)
        similarity = cos_sim(soru_emb, cevap_emb).item()
        similarities.append(similarity)
    
    avg_sim = np.mean(similarities)
    std_sim = np.std(similarities)
    min_sim = np.min(similarities)
    max_sim = np.max(similarities)
    
    print(f"  Ortalama benzerlik: {avg_sim:.4f} ± {std_sim:.4f}")
    print(f"  Min benzerlik: {min_sim:.4f}")
    print(f"  Max benzerlik: {max_sim:.4f}")
    print(f"  Median benzerlik: {np.median(similarities):.4f}")
    
    return avg_sim, similarities

# Orijinal model
print("\n🔄 Orijinal model test ediliyor...")
original_model = SentenceTransformer(MODEL_NAME)
original_score, original_sims = evaluate_model(original_model, test_df, "Orijinal BERT")

# Fine-tuned model
print("\n🔄 Fine-tuned model test ediliyor...")
finetuned_score, finetuned_sims = evaluate_model(finetuned_model, test_df, "Fine-tuned BERT")

# Karşılaştırma
improvement = ((finetuned_score - original_score) / original_score * 100)

print("\n" + "="*60)
print("📈 PERFORMANS KARŞILAŞTIRMASI")
print("="*60)
print(f"Orijinal Model:   {original_score:.4f}")
print(f"Fine-tuned Model: {finetuned_score:.4f}")
print(f"\n🎯 İyileşme: {improvement:+.2f}%")
print("="*60)

## 🔍 10. Örnek Sorgular ile Test

In [ ]:
def search_answer(query, knowledge_base_df, model, top_k=5):
    """
    Verilen soru için en iyi cevapları bulur.
    """
    # Query embedding
    query_embedding = model.encode(query, convert_to_tensor=True)
    
    # Tüm cevapların embeddingleri (batch processing)
    answers = knowledge_base_df['cevap'].tolist()
    answer_embeddings = model.encode(
        answers, 
        convert_to_tensor=True,
        batch_size=32,
        show_progress_bar=False
    )
    
    # Similarity hesapla
    scores = cos_sim(query_embedding, answer_embeddings)[0]
    
    # Top-k sonuçlar
    top_results = torch.topk(scores, k=min(top_k, len(knowledge_base_df)))
    
    results = []
    for score, idx in zip(top_results[0], top_results[1]):
        results.append({
            'skor': score.item(),
            'soru': knowledge_base_df.iloc[idx]['soru'],
            'cevap': knowledge_base_df.iloc[idx]['cevap']
        })
    
    return results

# Test soruları
test_queries = [
    "Yapay zeka ne işe yarar?",
    "Programlama dilleri hakkında bilgi ver",
    "Sinir ağları nasıl öğrenir?"
]

print("\n" + "="*80)
print("🔍 ÖRNEK SORGULAR")
print("="*80)

for query in test_queries:
    print(f"\n❓ SORU: {query}")
    print("-"*80)
    
    results = search_answer(query, df, finetuned_model, top_k=3)
    
    for i, result in enumerate(results, 1):
        print(f"\n{i}. Skor: {result['skor']:.4f}")
        print(f"   Soru: {result['soru']}")
        print(f"   Cevap: {result['cevap'][:150]}...")  # İlk 150 karakter

## 💾 11. Model Kaydetme ve Export

In [ ]:
# Model zaten OUTPUT_PATH'e kaydedildi
print("💾 Model Bilgileri:")
print(f"  Model path: {OUTPUT_PATH}")
print(f"  Model boyutu: {sum(p.numel() for p in finetuned_model.parameters()):,} parametreler")

# Model yükleme kodu
print("\n📝 Modeli yüklemek için:")
print(f"\nfrom sentence_transformers import SentenceTransformer")
print(f"model = SentenceTransformer('{OUTPUT_PATH}')")

# Hugging Face Hub'a yüklemek isterseniz:
print("\n☁️  Hugging Face Hub'a yüklemek için:")
print("\nfrom huggingface_hub import login")
print("login()  # Token girin")
print(f"finetuned_model.save_to_hub('kullanici-adi/model-adi')")

print("\n✅ Tüm işlemler tamamlandı!")

## 📈 12. Training Metrikleri Görselleştirme (Opsiyonel)

In [ ]:
# Training log dosyasını okuma ve görselleştirme
import matplotlib.pyplot as plt
import glob

# Log dosyasını bul
log_files = glob.glob(f"{OUTPUT_PATH}/**/eval/*.csv", recursive=True)

if log_files:
    print(f"📊 Training metrikleri görselleştiriliyor...")
    
    # İlk log dosyasını oku
    log_df = pd.read_csv(log_files[0])
    
    if 'spearman_cosine' in log_df.columns:
        plt.figure(figsize=(12, 5))
        
        # Subplot 1: Spearman Correlation
        plt.subplot(1, 2, 1)
        plt.plot(log_df['epoch'], log_df['spearman_cosine'], marker='o')
        plt.xlabel('Epoch')
        plt.ylabel('Spearman Correlation')
        plt.title('Validation Performance')
        plt.grid(True, alpha=0.3)
        
        # Subplot 2: Steps vs Performance
        plt.subplot(1, 2, 2)
        plt.plot(log_df['steps'], log_df['spearman_cosine'], marker='o', color='green')
        plt.xlabel('Steps')
        plt.ylabel('Spearman Correlation')
        plt.title('Training Progress')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'{OUTPUT_PATH}/training_metrics.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        print(f"✅ Grafik kaydedildi: {OUTPUT_PATH}/training_metrics.png")
else:
    print("⚠️  Log dosyası bulunamadı")

## 🎓 Kullanım Örnekleri ve Best Practices

### 🚀 Hızlı Kullanım:

```python
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

# Modeli yükle
model = SentenceTransformer('./turkish-bert-qa-finetuned_YYYYMMDD_HHMMSS')

# Soru ve cevap encode et
soru = "Python nasıl öğrenilir?"
cevap = "Python öğrenmek için önce temel syntax'ı öğrenmelisiniz..."

soru_emb = model.encode(soru)
cevap_emb = model.encode(cevap)

# Benzerlik hesapla
similarity = cos_sim(soru_emb, cevap_emb)
print(f"Benzerlik: {similarity.item():.4f}")
```

### 💡 İpuçları:

1. **Veri Kalitesi**: Daha fazla ve kaliteli veri = Daha iyi performans
2. **Negatif Örnekler**: Hard negative mining performansı artırır
3. **Batch Size**: GPU belleğinize göre optimize edin
4. **Epochs**: Çok fazla epoch overfitting'e yol açabilir
5. **Evaluation**: Düzenli olarak validation performansını kontrol edin

### 🔧 Hyperparameter Tuning:

```python
# Deneyebileceğiniz kombinasyonlar:
BATCH_SIZES = [8, 16, 32]
LEARNING_RATES = [2e-5, 3e-5, 5e-5]
EPOCHS = [2, 3, 4]
```

### 📊 Model Karşılaştırma:

Farklı loss fonksiyonları ve hiperparametreler deneyin:
- CosineSimilarityLoss
- MultipleNegativesRankingLoss
- ContrastiveLoss
- TripletLoss